# Day 8: Handheld Halting

Reference: https://adventofcode.com/2020/day/8

## Part 1

Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (acc, jmp, or nop) and an argument (a signed number like +4 or -20).

- `acc` increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
- `jmp` jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.
- `nop` stands for No OPeration - it does nothing. The instruction immediately below it is executed next.

For example, consider the following program:
```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```
These instructions are visited in this order:
```
nop +0  | 1
acc +1  | 2, 8(!)
jmp +4  | 3
acc +3  | 6
jmp -3  | 7
acc -99 |
acc +1  | 4
jmp -4  | 5
acc +6  |
```
First, the nop +0 does nothing. Then, the accumulator is increased from 0 to 1 (acc +1) and jmp +4 sets the next instruction to the other acc +1 near the bottom. After it increases the accumulator from 1 to 2, jmp -4 executes, setting the next instruction to the only acc +3. It sets the accumulator to 5, and jmp -3 causes the program to continue back at the first acc +1.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is 5.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

In [1]:
from IPython.display import Markdown
#from IPython.core.debugger import set_trace as breakpt

In [2]:
with open('code_input.txt', 'r') as fid:
#with open('test1_code_input.txt', 'r') as fid:
    stack = fid.read().splitlines()

#stack

In [3]:
def parse_cmd(cmdstr, ptr):
    cmd, val = cmdstr.split(' ')
    val = int(val)
    if 'acc' == cmd:
        accumval = val
        ptr += 1
    elif 'jmp' == cmd:
        accumval = 0
        ptr += val
    elif 'nop' == cmd:
        accumval = 0
        ptr += 1
    else:
        raise ValueError("Unhandled cmd '{}''".format(cmd))
    return (ptr, accumval)

accum = 0
ptr = 0
ptrs_seen = set()
ptr = 0
while(True):
    #print(ptr, accum, stack[ptr])
    ptr, accumval = parse_cmd(stack[ptr], ptr)
    if ptr in ptrs_seen:
        #print("Already seen ptr: {}".format(ptr))
        #print("Accumulator value: {}".format(accum))
        break
    if accumval:
        accum += accumval
    ptrs_seen.add(ptr)
    

In [4]:
#Markdown("Accumulator before program hang: **{}**".format(accum))

## Part 2

After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a jmp is supposed to be a nop, or a nop is supposed to be a jmp. (No acc instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one jmp or nop, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:
```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```
If you change the first instruction from nop +0 to jmp +0, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from jmp -4 to nop -4), the program terminates! The instructions are visited in this order:
```
nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6
```
After the last instruction (acc +6), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (acc +1, acc +1, acc +6).

Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the value of the accumulator after the program terminates?

In [5]:
# Convert stack to 2 lists of ops and vals
stack_ops, stack_params = list(zip(*[x.split(' ') for x in stack]))
stack_ops = list(stack_ops)
stack_vals = [int(x) for x in stack_params]

In [6]:
def parse_op(cmd, val, ptr):
    if 'acc' == cmd:
        accumval = val
        ptr += 1
    elif 'jmp' == cmd:
        accumval = 0
        ptr += val
    elif 'nop' == cmd:
        accumval = 0
        ptr += 1
    else:
        raise ValueError("Unhandled cmd '{}''".format(cmd))
    return (ptr, accumval)

def test_hang(stack_ops, stack_vals):
    # Program succeeds if it gets to the last
    # instruction at ptr_end in ehich case return the
    # accumulator values. Otherwise return None
    ptr_end = len(stack_ops) - 1
    accum = 0
    ptr = 0
    ptrs_seen = set()
    ptr = 0
    while(True):
        #print(ptr, accum, stack[ptr])
        ptr, accumval = parse_op(stack_ops[ptr], stack_vals[ptr], ptr)
        if ptr in ptrs_seen:
            #print("Already seen ptr: {}".format(ptr))
            #print("Accumulator value: {}".format(accum))
            # Program hangs
            #return [None, ptr, accum]
            return None
        if accumval:
            accum += accumval
        if ptr == ptr_end:
            # Got to the end. Success!
            return accum
        ptrs_seen.add(ptr)


#test_hang(stack_ops, stack_vals)

In [7]:
# Process stack_ops each time swapping a jmp for a nop or vice versa
accum = None
for i, op in enumerate(stack_ops):
    if 'nop' == op:
        so = stack_ops.copy()
        so[i] = 'jmp'
    elif 'jmp' == op:
        so = stack_ops.copy()
        so[i] = 'nop'
    else:
        # Nothing to try changing
        continue
    accum = test_hang(so, stack_vals)
    if accum is not None:
        print("SUCCESS!")
        break

SUCCESS!


In [8]:
#Markdown("Accumulator for successful program run: **{}**".format(accum))